# Assignment 2: Slack Bot
### Max Davish | PPDS Spring 17

For this assignment, I've created a bot that, when asked, will return information about the current weather in a given citb y querying the Open Weather Map API. The annotated code from the script I used is below. 

In [2]:
#!/usr/bin/python3

#The various libraries we'll have to use:
import json
import time
import re
import requests

In [12]:
secrets_file = 'slack_secret.json'
    #Copied via terminal from NYU_Notes over to this folder.
f = open(secrets_file, 'r') 
content = f.read()
auth_info = json.loads(content)
token = auth_info["access_token"]
    #So we can authorize the SlackClient library in the next cell.
f.close()

In [13]:
from slackclient import SlackClient
sc = SlackClient(token)

In [6]:
#So we can recognize when our bot is queried:
#Recognizes messages containing my username (@maxdavish) followed by the word "bot"
def message_matches(user_id, message_text):
    regex_expression = '.*@' + user_id + '.*bot.*'
    regex = re.compile(regex_expression)
    match = regex.match(message_text)
    return match != None 

In [7]:
#This function uses regular expressions to extract the name of a city a query with the following structure:
    #How's the weather in London?
#It accepts cities with multiple words (e.g. New York, Hong Kong) and mostly ignores capitalization.
def Extract_CityName(message_text):
    regex_expression = '[H,h]ow\'s the weather in ([A-z, ]+)\?'
    regex = re.compile(regex_expression)
    matches = regex.finditer(message_text)
    for match in matches:
        return match.group(1)
    
    return None

In [14]:
#This code is recycled from assignment 1; it queries the Open Weather API.
key = '097217bc8c67dcadb7f7f7b295429be1'
def Get_WeatherData(key, city):
    import requests
    return requests.get('http://api.openweathermap.org/data/2.5/weather?q={city}&appid={key}'.format(city=city,key=key)).json()

In [9]:
#For some reason, the Open Weather API returns kelvin temperatures. This formula converts them to farneheit. 
def Kelvin_to_Farenheit(temp_kelvin):
    return (temp_kelvin - 273) * (9/5) + 32

In [10]:
#This function dictates the bot's response:
    #If the Extract_CityName function offers no response, the bot asks the user to rephrase the query altogether.
    #If the Extract_CityName function produces a city that the API doesn't recognize, the bot prompts the user to retype the city.
    #If all goes well, the bot simply tells the user the temperature and status of the weather in the relevant city. 
def Return_WeatherResults(city, user):
    key = '097217bc8c67dcadb7f7f7b295429be1'
    data = Get_WeatherData(key, city)
    
    if city == None:
        return "I'm sorry @{user}, I'm not sure I understand. Try rephrasing in this form: \n \'Hey @maxdavish bot, how's the weather in London?\'".format(user=user)
    
    if 'message' in data.keys():
        return "I'm sorry @{user}, I'm not familiar with that place. Please try again.".format(user=user)

    else: 
        city = city
        temperature = round(Kelvin_to_Farenheit(data['main']['temp']), 1)
        hightemp = round(Kelvin_to_Farenheit(data['main']['temp_max']), 1)
        lowtemp = round(Kelvin_to_Farenheit(data['main']['temp_min']), 1)
        description = data['weather'][0]['description']
        result = "@{user}, currently in {city} it's {temperature} degrees farenheit with {description}, with a high of {hightemp} and a low of {lowtemp}.".format(user=user, city=city,temperature=temperature,description=description,hightemp=hightemp,lowtemp=lowtemp)
        return result


In [ ]:
#This code is shamelessly copied from Panos's CitiBike example:
if sc.rtm_connect():
    while True:
        time.sleep(1)
        response = sc.rtm_read()
        for item in response:
            if item.get("type") != 'message':
                continue
                
            if item.get("user") == None:
                continue
            
            user_id = auth_info["user_id"]
            message_text = item.get('text')
            if not message_matches(user_id, message_text):
                continue
                
            response = sc.api_call("users.info", user=item["user"])
            username = response['user'].get('name')
        
            city_name = Extract_CityName(message_text)

            message = Return_WeatherResults(city_name, username)

            sc.api_call("chat.postMessage", channel="#assignment2_bots", text=message)